In [ ]:
import time
import os
from google import genai
import pandas as pd
from google.genai import types
file_path = 'Dataset/2_label/V1_5 divisions_per_second.xlsx'
sheet_name = 'Metadata'

df = pd.read_excel(file_path, sheet_name=sheet_name)
df.set_index('Column (Header)', inplace=True)

gauge_type = df.loc['type', 'Content / Example']
unit = df.loc['unit', 'Content / Example']
reading_start = df.loc['manufacturer_range_min', 'Content / Example']
reading_end = df.loc['manufacturer_range_max', 'Content / Example']

reading_start = float(reading_start)
reading_end = float(reading_end)
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
graduation_interval = df.loc['graduation_interval', 'Content / Example']
model_name = "gemini-3-flash-preview"
# model_name = "gemini-3-pro-preview"
VIDEO_DIR = 'Dataset/2.- Analog Depth Gauge/'
video_files = [f for f in os.listdir(VIDEO_DIR) if f.endswith('.mp4')]
output_dir = f'Results/2.- Analog Depth Gauge{model_name}_naive'
os.makedirs(output_dir, exist_ok=True)
    
for video_filename in video_files:
    seq_id = os.path.splitext(video_filename)[0]
    
    output_file = os.path.join(output_dir, f'{seq_id}_Raw_Results.xlsx')
    if os.path.exists(output_file):
        print(f"Sikpping: {seq_id} (the result file already exists)")
        continue
    
    print(f"\n{'='*20} Processing: {seq_id} {'='*20}")

    print("Uploading...")
    program_start_time = time.time()
    myfile = client.files.upload(file=f"{VIDEO_DIR}{seq_id}.mp4")
    print(f"Uploaded: {myfile.name}")


    print("Processing...")
    while True:

        file_status = client.files.get(name=myfile.name)

        if file_status.state == "ACTIVE":
            print("Succeed")
            break
        elif file_status.state == "FAILED":
            raise Exception("Failed (State: FAILED)")

        print("Processing, waiting...")
        time.sleep(5)

    
# CoT prompt
#     message = f'''
#     ROLE: Expert Industrial Metrology Assistant

#     PROTOCOL: See-Think-Confirm
#     1. SEE: Localize gauge boundaries. Identify [0%] and [100%] markers.
#     2. THINK: Synchronize needle position with the in-band digital clock. Calculate velocity (ΔReading/ΔTime). Use 'Thought Signatures' to maintain temporal state between frames.
#     3. CONFIRM: Verify that the reading is within [min, max] and follows physical monotonicity relative to previous frames.
#     CONSTRAINT: Output JSON only. Format: {{"ts_ms": integer, "reading": float, "confidence": float}}
#     TASK: Extract a high-precision time-series of readings from the video.

#     INSTRUMENT METADATA:
#     - Gauge Type:{gauge_type} | Unit: {unit} 
#     - Graduation Interval: {graduation_interval}
#     - Calibrated Range: [{reading_start}, {reading_end}]

#     SAMPLING PROTOCOL:
#     - Absolute Time Reference: Every 'ts_ms' in your output must correspond to the EXACT numerical value shown on the digital chronometer in the frame.
#     - Starting Point: The chronometer in this video starts at a non-zero value. Ignore any frames before the timer reaches the first 200ms integer increment (e.g., if the timer starts at 5670ms, your first entry should be at 5800ms).
#     - Sampling Interval: Provide one reading every 200ms based on the chronometer's increments (e.g., 5800, 6000, 6200...). 
#     - Duration: Continue this sequence until the video ends, strictly following the chronometer's values, regardless of the video file's elapsed time.

#     RESPONSE REQUIREMENT:
#     Output a single JSON array where each entry contains: {{"ts_ms": integer, "reading": float, "confidence": float}}
#     '''


# naive prompt
    message = f'''
    ROLE: Expert Industrial Metrology Assistant

    TASK: Read the analog gauge values from the video at specific time intervals.

    INSTRUMENT METADATA:
    - Gauge Type:{gauge_type} | Unit: {unit} 
    - Graduation Interval: {graduation_interval}
    - Calibrated Range: [{reading_start}, {reading_end}]

    SAMPLING REQUIREMENTS:
    - Use the digital chronometer shown in the video for timing.
    - Start reading from the first 200ms integer increment.
    - Provide one reading every 200ms (e.g., 5800, 6000, 6200...).
    - Continue until the video ends.

    OUTPUT FORMAT:
    Return only a JSON array of objects: {{"ts_ms": integer, "reading": float, "confidence": float}}
    '''
    print("Generating...")


    response = client.models.generate_content(
        model=model_name, 
        contents=[myfile, message],
        config=types.GenerateContentConfig(
            temperature=0.0,
        )
    )
    # "Identify the analog gauge and output a timestamped table of its readings based on the visible dial markings."
    print(response.text)


    program_end_time = time.time()
    total_duration = program_end_time - program_start_time
    print(total_duration)

    df_output = pd.DataFrame([{
        "sequence_id": seq_id,
        "raw_model_response": response.text,
        "total_duration_sec": round(total_duration, 2),
        "model_name": model_name,
    }])

    df_output.to_excel(output_file, index=False)